## Data Cleaning

Using a test run dataset, I show how json data was extracted and converted into read dataframes

In [8]:
import json
import pandas as pd

In [9]:
with open('data/test-run.json', 'r') as file:
    data = json.load(file)

In [10]:
data

[{'number_of_runs': 2, 'number_of_simulation_iterations': 500},
 {'final_bankroll': 70,
  'rounds_played': 5,
  'initial_bankroll': 100,
  'round_outcomes': [5, -5, -5, -5, 5]},
 {'final_bankroll': 50,
  'rounds_played': 5,
  'initial_bankroll': 100,
  'round_outcomes': [-5, -5, -5, -5, -5]}]

In [11]:
df = pd.read_json('data/test-run.json')
df

,number_of_runs,number_of_simulation_iterations,final_bankroll,rounds_played,initial_bankroll,round_outcomes
0,2.0,500.0,NaN,NaN,NaN,NaN
1,NaN,NaN,70.0,5.0,100.0,"[5, -5, -5, -5, 5]"
2,NaN,NaN,50.0,5.0,100.0,"[-5, -5, -5, -5, -5]"


In [12]:
metadata = df[['number_of_runs', 'number_of_simulation_iterations']].dropna()
metadata

,number_of_runs,number_of_simulation_iterations
0,2.0,500.0


In [13]:
final_results = df[['final_bankroll', 'rounds_played', 'initial_bankroll']].dropna()
final_results

,final_bankroll,rounds_played,initial_bankroll
1,70.0,5.0,100.0
2,50.0,5.0,100.0


In [14]:
game_data = df[['round_outcomes']].dropna().reset_index(names='game')

# Explode the 'round_outcomes' column to separate each outcome into its own row
game_data = game_data.explode('round_outcomes').reset_index(drop=True)

# Add round numbers for each game
game_data['round'] = game_data.groupby('game').cumcount() + 1

game_data


,game,round_outcomes,round
0,1,5,1
1,1,-5,2
2,1,-5,3
3,1,-5,4
4,1,5,5
5,2,-5,1
6,2,-5,2
7,2,-5,3
8,2,-5,4
9,2,-5,5


## Exploratory Analysis

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot()